Welcome to the Outbreak Prediction Colab Notebook!


Each section will include references for the materials used to develop this notebook. The data used here is up to date as of the last time the spreadsheet was updated, and all other information is up to date as of 2020.

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime
from datetime import datetime


We start by loading the data, organizing it and fianlly plotting it.
The dataset that is being used from the European CDC,  but has been compiled into an easier to use spreadsheet.

In [ ]:
current_data.head()


In [ ]:
# Here, we load the data directly from the website. 
#https://ourworldindata.org/coronavirus-source-data

current_data = pd.read_csv('https://covid.ourworldindata.org/data/jhu/full_data.csv') 
current_data_array = np.array(current_data)

# Now, we want to get a list of the countries in the spreadsheet to make it easier to visualize our data. 

locations = np.array(current_data['location'])
country_list = list(dict.fromkeys(locations))
print(country_list)

# To visualize the data from a country of our choice, we need to find the location of that data in the spreadsheet, and save the information to a new location! 

country_ind = country_list.index('United States') # Type in the name of your country here!
new_ind = current_data_array[:,1] == country_list[country_ind]

new_cases = current_data_array[new_ind,2]
new_deaths = current_data_array[new_ind,3]
total_cases = current_data_array[new_ind,4]
total_deaths = current_data_array[new_ind,5]

dates = np.linspace(1, len(new_cases), len(new_cases)) # Because the US has updated daily, we can do this.
 #dates = current_data_array[new_ind,0] 

#For other countries where updates do not occur daily. 

# Finally, we'll start by visualizing the data from our country of choice. 
plt.figure(figsize=(8, 8))
plt.plot(dates, new_cases, dates, new_deaths)
plt.legend(['New Cases', 'New Deaths'])

plt.figure(figsize=(8,8))
plt.plot(dates, total_cases, dates, total_deaths)
plt.legend(['Total Cases', 'Total Deaths'])
plt.show()


Now that we can see the data from the past, let's try to see what things will look like in the future! To do this, we'll be indirectly using an approach called logistic regression to model exponential growth.
Using population data from the US, we'll look at how coronavirus might spread without any intervention. To do this, we'll focus on the data on new cases, and will look at how the number of new cases compares to the available medical resources. We didn't model deaths here.
In order to figure out the exact exponential formula, we apply the natural logirithm to the number of new cases over time to get a fairly linear line. This makes it easier to find our formula - all we need to find is the slope!

From there, we can model the actual growth by multplying the number of days it's been since we started recording outbreaks by the slope, and then take the exponential of that number. You can see all this below!

Now, this approach applies to a point but has several limitations - as fewer people remain healthy, there are fewer people to infect, and the disease spread plateaus. 

In [ ]:

import sklearn
from sklearn.linear_model import LinearRegression

dates = np.linspace(1, len(new_cases), len(new_cases)) 
dates = dates[:,np.newaxis] #dates from the US cases 

log_cases = np.log(np.array(new_cases, dtype=np.float64)) # Taking the natural log of the number of new cases in order to create a linear plot. 
infs = np.isinf(log_cases) # Setting places where the number of cases was 0 equal to 0 on the log scale - natural log of zero is negative infinity, which won't work with our plots.
log_cases[infs] = 0

model =  LinearRegression().fit(dates[60:],log_cases[60:]) # Performing a linear regression on the section of the data with new reported cases (Days 60+) 
pred = model.predict(dates) # Predicting the natural log of the number of cases based on the known dates.

dates_into_future = np.linspace(1, 90, 90) # Now, we want to look at future dates
fut_pred = model.predict(dates_into_future[60:, np.newaxis]) # Making predictions in future dates

plt.figure(figsize=(12, 10)) # Here, we're plotting the natural log of the new cases and the linear model that we've fit to that data. 
plt.plot(dates,  log_cases, 'o', dates[60:], pred[60:])
plt.legend(['ln(New Cases) (Actual)', 'ln(New Cases) (Linear Fit)'])
plt.show()

plt.figure(figsize=(12, 10)) # Here, we're plotting the actual data and the projection of the next 10 days (Days 81-90) from the linear fit that we made before. 
plt.plot(dates,  new_cases, 'o', dates_into_future[60:], np.exp(fut_pred))
plt.legend(['New Cases (Actual)', 'New Cases (Projected from Linear Fit)'])
plt.show()


***Exponential growth*** **is a useful model for looking at how disease will spread if left unattended, but doesn't factor in a few important details:**



There are only so many people in the world, so eventually, the number of new cases has to plateau.
People get better! Exponential growth assumes that everyone who catches a disease has it forever, but as we already know, most people end up recovering from coronavirus and becoming either temporarily or permanently immune*. This means that, at some point, this number of people with the disease has to go down.
The rate at which the disease spreads can be changed. This is where things like Flattening The Curve and Social Distancing come in - if you're not around other people, than the replication rate of the disease decreases because you physically can't spread it.
*We don't currently know how long someone is immune to coronavirus after recovering from it.


Another model that we can use to address these assumptions is the ***SIR model***. This model assumes that there is a certain population of people who can catch the disease (S), a population of people who are infected (I), and a population of people who are recovered and have immunity (R).

This model relies on solving a set of differential equations, which I'm not going to get into the details of, but in short:

The number of people who can catch the disease decreases based on their contact with those infected.
The number of people infected increases and decreases based on the number of people they have contact with and how long it takes to recover.
The number of people who are immune increases based on the time it takes to recover when infected.
This is a better model, but it also makes some assumptions and omits some important details. Namely, it doesn't account for death. The total population does not change, just the ratio of the number of susceptible, infected, and immune people.

For the purposes of this example, I also make a few assumptions:

-The recovery rate is based on the assumption that it takes about two weeks to recover from the disease AND that a person who has recovered cannot spread it to others. We don't know if this is actually true.

-The contact rate describes the number of people that an infected person comes into contact with per day. This value varies based on things like Flattening The Curve and Social Distancing - if you're not around other people, the numbeer goes down, and the disease spreads less.

-The contact rate used here is commonly used for the flu, but we don't really have a contact rate that reflects both how many people you come in contact with and how infectious the disease is for coronavirus. In other words, this shows the relative effect of social distancing on flattening the curve, but these values should not be taken as fact. 


In [ ]:
import scipy
from scipy.integrate import odeint

population = 327200000 # Population of the United States

initial_infected, initial_recovered = 1, 0 # Initial conditions for infected and recovered people

initial_everyone_else = population - initial_infected - initial_recovered # Initial conditions for everyone else. 

initial_conditions = initial_everyone_else, initial_infected, initial_recovered 

n_days = 500 # Days over which to integrate 
time = np.linspace(0, n_days, n_days)

contact_rate = 0.25 # Contact Rate - We don't know this for coronavirus, so use it as a relative term for comparison. 
recovery_rate = 1/14 # Recovery Rate - 

# The SIR model, integreated over 500 days. 

def SIR(initial_conditions, t, population, contact_rate, recovery_rate):
  S, I, R = initial_conditions 
  dS = -contact_rate*S*I/population
  dI = contact_rate*S*I/population - recovery_rate*I 
  dR = recovery_rate*I 
  return dS, dI, dR

result = odeint(SIR, initial_conditions, time, args=(population, contact_rate, recovery_rate))
S, I, R = result.T

fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(time, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(time, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(time, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Days')
ax.set_ylabel('Number (1000s)')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)

fig = plt.figure(facecolor='w', figsize=(9, 10))
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(time, S/1000, 'b', alpha=0.5, lw=10, label='Susceptible')
ax.plot(time, I/1000, 'r', alpha=0.5, lw=10, label='Infected')
ax.plot(time, R/1000, 'g', alpha=0.5, lw=10, label='Recovered with immunity')
ax.set_xlabel('Days')
ax.set_ylabel('Number (1000s)')


# What happens if we're in contact with fewer people? 
contact_rate = 0.15

result = odeint(SIR, initial_conditions, time, args=(population, contact_rate, recovery_rate))
S, I, R = result.T

fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, axisbelow=True)
ax.plot(time, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(time, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(time, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Days')
ax.set_ylabel('Number (1000s)')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
plt.show()


The SIR epidemic model
A simple mathematical description of the spread of a disease in a population is the so-called SIR model, which divides the (fixed) population of N individuals into three "compartments" which may vary as a function of time, t:

S(t) are those susceptible but not yet infected with the disease;
I(t) is the number of infectious individuals;
R(t) are those individuals who have recovered from the disease and now have immunity to it.
The SIR model describes the change in the population of each of these compartments in terms of two parameters, β and γ. β describes the effective contact rate of the disease: an infected individual comes into contact with βN other individuals per unit time (of which the fraction that are susceptible to contracting the disease is S/N). γ is the mean recovery rate: that is, 1/γ is the mean period of time during which an infected individual can pass it on.

dSdtdIdtdRdt=−βSIN,=βSIN−γI,=γI.

The following Python code integrates these equations for a disease characterised by parameters β=0.2, 1/γ=10days in a population of N=1000 (perhaps 'flu in a school). The model is started with a single infected individual on day 0: I(0)=1.
The plotted curves of S(t), I(t) and R(t) are styled to look a bit nicer than Matplotlib's defaults.

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

# Total population, N.
N = 1000
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0 = 1, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 0.2, 1./10 
# A grid of time points (in days)
t = np.linspace(0, 160, 160)

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

# Initial conditions vector
y0 = S0, I0, R0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

# Plot the data on three separate curves for S(t), I(t) and R(t)
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
ax.plot(t, S/1000, 'b', alpha=0.5, lw=2, label='Susceptible')
ax.plot(t, I/1000, 'r', alpha=0.5, lw=2, label='Infected')
ax.plot(t, R/1000, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
ax.set_xlabel('Time /days')
ax.set_ylabel('Number (1000s)')
ax.set_ylim(0,1.2)
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend()
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show()